## _*HDF5 files and HDF5 driver*_

Qiskit Chemistry supports a number of different chemistry drivers, i.e chemistry programs and software libraries, which are used to compute integrals that are then used to build the second quantized Hamiltonian in the FermionicOperator.

Drivers, built using the programs and libraries, include those for Gaussian 16, PyQuante, PySCF and PSI4. The main Qiskit documentation has more information on [drivers](https://qiskit.org/documentation/aqua/chemistry/qiskit_chemistry_drivers.html).

When a driver is run, Qiskit Chemistry outputs the result in a common format for later processing. This output, the `QMolecule`, includes electron integrals, numbers of electrons, molecular orbital values and so on. While drivers can output different values, even for the same problem, due to computational differences, the result is still in this common format.

This QMolecule then is the output of the drivers and input to the rest of the chemistry stack. Now we have a capability in the QMolecule to be save it as a file in [HDF5](https://www.hdfgroup.org/solutions/hdf5/) format. We can also load such a saved HDF5 file back into a QMolecule to re-create the original. The latter capability we built into a driver we call the HDF5 driver. This driver the file and presents a QMolecule to the chemistry stack just as the other drivers.

Let's take a look at the HDF5 driver and how to create HDF5 files.

### HDF5 driver

This tutorials folder has some HDF5 that were saved before. So let's first use the HDF5 driver to load one up and output a QMolecule.

The HDF5 file name tell us it was an H2 molecule, at interatomic distance 0.735 with a basis set of sto-3g that came from the original driver. _Note: this naming convention is just what is used by us in the HDF5 samples here, no naming convention is enforced._

We'll print some fields from the QMolecule, that are set from the file, to show a small part of its content, which indeed matches what we would expect.

In [1]:
from qiskit.chemistry.drivers import HDF5Driver

driver = HDF5Driver('./h2_0.735_sto-3g.hdf5')
molecule = driver.run()

print('Number of orbitals: {}'.format(molecule.num_orbitals))
print('Number of alpha electrons: {}'.format(molecule.num_orbitals))
print('Number of beta electrons: {}'.format(molecule.num_orbitals))

Number of orbitals: 2
Number of alpha electrons: 2
Number of beta electrons: 2


To show this molecule can now be used as input to the rest of the chemistry we will compute its ground state energy. This is the electronic part that is computed. As the focus of this tutorial is around the HDF5 file and driver I am not going to explain the code below. There are other tutorials that cover this aspect. Also this uses a classical algorithm from Aqua just to keep things simpler. Again other tutorials here cover this ground state energy problem showing VQE or IQPE to solve it.

In [2]:
from qiskit.chemistry.core import Hamiltonian, TransformationType, QubitMappingType 
from qiskit.aqua.algorithms.classical import ExactEigensolver

core = Hamiltonian(transformation=TransformationType.FULL, qubit_mapping=QubitMappingType.PARITY, 
                   two_qubit_reduction=True)
qubit_op, aux_ops = core.run(molecule)

ee = ExactEigensolver(qubit_op, aux_operators=aux_ops)
result = ee.run()

print(result['energy'])

-1.8572750270315876


Another field in the QMolecule is the nuclear repulsion energy. We can combine this with above result to compute the total ground state energy

In [3]:
print(result['energy'] + molecule.nuclear_repulsion_energy)

-1.137306035752676


The chemistry stack can produce a formatted result from the algorithms result. I show it here for comparison to above output.

In [4]:
lines, full_result = core.process_algorithm_result(result)
print(*lines, sep='\n')

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.857275027032
  - computed part:      -1.857275027032
  - frozen energy part: 0.0
  - particle hole part: 0.0
~ Nuclear repulsion energy (Hartree): 0.719968991279
> Total ground state energy (Hartree): -1.137306035753
  Measured:: Num particles: 2.000, S: 0.000, M: 0.00000
 
=== DIPOLE MOMENT ===
 
* Electronic dipole moment (a.u.): [0.0  0.0  0.0]
  - computed part:      [0.0  0.0  0.0]
  - frozen energy part: [0.0  0.0  0.0]
  - particle hole part: [0.0  0.0  0.0]
~ Nuclear dipole moment (a.u.): [0.0  0.0  0.0]
> Dipole moment (a.u.): [0.0  0.0  0.0]  Total: 0.0
               (debye): [0.0  0.0  0.0]  Total: 0.0


#### Declarative use of HDF5 driver

We can easily use an HDF5 file with the declarative dictionary approach too. 

In [5]:
from qiskit.chemistry import QiskitChemistry

qiskit_chemistry_dict = {
    'driver': {'name': 'HDF5'},
    'HDF5': {'hdf5_input': './h2_0.735_sto-3g.hdf5'},
    'operator': {'name': 'hamiltonian', 'qubit_mapping': 'parity'},
    'algorithm': {'name': 'ExactEigensolver'}
}

solver = QiskitChemistry()
result = solver.run(qiskit_chemistry_dict)
    
print(*result['printable'], sep='\n')    

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.857275027032
  - computed part:      -1.857275027032
  - frozen energy part: 0.0
  - particle hole part: 0.0
~ Nuclear repulsion energy (Hartree): 0.719968991279
> Total ground state energy (Hartree): -1.137306035753
  Measured:: Num particles: 2.000, S: 0.000, M: 0.00000
 
=== DIPOLE MOMENT ===
 
* Electronic dipole moment (a.u.): [0.0  0.0  0.0]
  - computed part:      [0.0  0.0  0.0]
  - frozen energy part: [0.0  0.0  0.0]
  - particle hole part: [0.0  0.0  0.0]
~ Nuclear dipole moment (a.u.): [0.0  0.0  0.0]
> Dipole moment (a.u.): [0.0  0.0  0.0]  Total: 0.0
               (debye): [0.0  0.0  0.0]  Total: 0.0


#### HDF5 file

Lets look at creation of the HDF5 file. There can be many reasons you might want to do this, for example to ensure you are always testing from the exact same content.

Lets first create a QMolecule instance using the PySCF driver. The same can be done with other drivers too.

In [6]:
from qiskit.chemistry.drivers import PySCFDriver, UnitsType

driver = PySCFDriver(atom='H .0 .0 .0; H .0 .0 0.735', unit=UnitsType.ANGSTROM,
                     charge=0, spin=0, basis='sto3g')
molecule = driver.run()

print('Number of orbitals: {}'.format(molecule.num_orbitals))
print('Number of alpha electrons: {}'.format(molecule.num_orbitals))
print('Number of beta electrons: {}'.format(molecule.num_orbitals))

Number of orbitals: 2
Number of alpha electrons: 2
Number of beta electrons: 2


Here we save the molecule, in the HDF5 format, by passing in a file name to the save() method. We will use here a temporary file (and delete it once we are done) but it's good practice to name the files after what it represents, such as the example hdf5 file used above, if you intend to save these for future use.

In [7]:
import os, tempfile

fd, hdf5_file = tempfile.mkstemp(suffix='.hdf5')
os.close(fd)

molecule.save(hdf5_file)
print('{} : {} bytes'.format(hdf5_file, os.path.getsize(hdf5_file)))

/tmp/tmpsqlhszxm.hdf5 : 15664 bytes


With the HDF5 file we can now re-create the original QMolecule by passing the file to the HDF5 driver just as we did in the above section.

In [8]:
driver = HDF5Driver(hdf5_file)
molecule1 = driver.run()

print('Number of orbitals: {}'.format(molecule1.num_orbitals))
print('Number of alpha electrons: {}'.format(molecule1.num_orbitals))
print('Number of beta electrons: {}'.format(molecule1.num_orbitals))

Number of orbitals: 2
Number of alpha electrons: 2
Number of beta electrons: 2


In [9]:
os.remove(hdf5_file)

#### Declarative approach and creating hdf5 file

Again we use a temporary file that we will delete when done. Here we specify the the file name of as the value of the `hdf5_output` key `driver` section. `QiskitChemistry` will see this field and simply run the driver and save the result (any algorithm in the dictionary will be ignored).

In [10]:
fd, hdf5_file = tempfile.mkstemp(suffix='.hdf5')
os.close(fd)

qiskit_chemistry_dict = {
    'driver': {'name': 'PYSCF', 'hdf5_output': hdf5_file},
    'PYSCF': {'atom': 'H .0 .0 .0; H .0 .0 0.735', 'basis': 'sto3g'},
}

solver = QiskitChemistry()
result = solver.run(qiskit_chemistry_dict)
    
print(*result['printable'], sep="\n") 

print('\n{} : {} bytes'.format(hdf5_file, os.path.getsize(hdf5_file)))

HDF5 file saved '/tmp/tmpvfryu2md.hdf5'

/tmp/tmpvfryu2md.hdf5 : 15664 bytes


In [11]:
driver = HDF5Driver(hdf5_file)
molecule = driver.run()

print('Number of orbitals: {}'.format(molecule.num_orbitals))
print('Number of alpha electrons: {}'.format(molecule.num_orbitals))
print('Number of beta electrons: {}'.format(molecule.num_orbitals))

Number of orbitals: 2
Number of alpha electrons: 2
Number of beta electrons: 2


In [12]:
os.remove(hdf5_file)

##### Saving sets of HDF5 files

Whether using the programmatic approach, or declarative approach, the above code can be used in a loop updating values to create a set of HDF5 files. For example you might want a set that span of a set of inter-atomic distances for a dissociation curve, or the same molecule with different basis sets. You can see many examples in the tutorials of looping over values to plot dissociation curves etc., so I am not going to show more code here. But this is simple to do knowing about the basics above. 